In [1]:
import openai
import json

In [2]:
# Add relevant functions
def get_bank_names(name):
    """Return given bank names"""

    result = {
        "name": name,
    }
    return json.dumps(result)

    
def find_period(start_year, end_year):
    """Extract a period from a sentence"""

    result = {
        "start_year": start_year,
        "end_year": end_year,
    }
    return json.dumps(result)


def get_conversation_category(risk_category):
    """Return a risk category: PD, LGD, EAD, RWA"""

    result = {
        "name": risk_category,
    }
    return json.dumps(result)
    

def run_conversation(user_input, call_type="auto"):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": user_input}]
    functions = [
        {
            "name": "get_bank_names",
            "description": "Extract bank name from the sentence",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "something like Deutsche bank, Goldman Sachs",
                    },
                },
                "required": ["name"],
            },
        },

        {
            "name": "find_period",
            "description": "Extract period (start year and end year) from the sentence",
            "parameters": {
                "type": "object",
                "properties": {
                    "start_year": {
                        "type": "string",
                        "description": "Start year of the period",
                    },

                    "end_year": {
                        "type": "string",
                        "description": "End year of the period",
                    },
                },
                "required": ["start_year", "end_year"],
            },
        },

        {
            "name": "get_conversation_category",
            "description": "Return one of the risk categories: PD, LGD, EAD, RWA",
            "parameters": {
                "type": "object",
                "properties": {
                    "risk_category": {
                        "type": "string",
                        "description": "Only one of the option is permitted: PD, LGD, EAD, RWA",
                    },
                },
                "required": ["risk_category"],
            },
        }
        
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call=call_type,
    )
    response_message = response["choices"][0]["message"]
    print(f'Answer returned by functions: {response["choices"][0]["message"]["function_call"]["arguments"]}\n')

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_bank_names": get_bank_names,
            "find_period": find_period,
            "get_conversation_category": get_conversation_category,
        }
        
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(**function_args)

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response

In [3]:
user_input = "the period started in 1998 and ended in 2010."
out = run_conversation(user_input)
print(f"Final answer: {out['choices'][0]['message']['content']}\n")

Answer returned by functions: {
  "start_year": "1998",
  "end_year": "2010"
}/n
Final answer: The period you specified, from 1998 to 2010, lasted for 12 years.



In [4]:
user_input = "i want to extract bank names from the following sentence: Deutsche Bank and Swedbank?"
out = run_conversation(user_input)
print(f"Final answer: {out['choices'][0]['message']['content']}\n")

Answer returned by functions: {
  "name": "Deutsche Bank and Swedbank"
}/n
Final answer: The bank names extracted from the sentence "Deutsche Bank and Swedbank" are "Deutsche Bank" and "Swedbank".



In [5]:
user_input = "I would like to discuss LGD modelling"
out = run_conversation(user_input, {"name": "get_conversation_category"})
print(f"Final answer: {out['choices'][0]['message']['content']}\n")

Answer returned by functions: {
  "risk_category": "LGD"
}/n
Final answer: Sure! LGD stands for Loss Given Default, and it is a measure used in credit risk modeling to assess the potential loss that may occur if a borrower defaults on their obligations. LGD modeling is the process of estimating and predicting the percentage of exposure that will be lost in the event of default.

LGD modeling involves analyzing various factors that contribute to the actual recovery rate in the event of default. These factors can include collateral value, seniority of the debt, loan characteristics, economic conditions, and recovery processes.

The purpose of LGD modeling is to aid financial institutions and lenders in quantifying and managing credit risk. By understanding the potential loss that may be incurred, lenders can set appropriate pricing, make informed lending decisions, and allocate capital effectively.

There are various methods used in LGD modeling, such as statistical modeling, historical 